# Creating own Chatbot

<img src="CC.png" width=400/>

* In my CIS 3990, Introduction to Artificial Intelligence class, we learned to create our own chatbots. 
* This chatbots capability is to convert one currency to another and would covert an amount you present to it. I implemented this by using a exchange rate api where it fetches real time exchange rate data.

--------------------------------------------------------------
import requests <br>
import json <br>
from openai.types.chat.chat_completion import ChatCompletionMessage <br>


def convert_currency(amount, from_currency, to_currency): <br>
    api_url = f"https://api.exchangerate-api.com/v4/latest/{from_currency.upper()}" <br>
    response = requests.get(api_url) <br>
    data = response.json() <br>

if "rates" not in data or to_currency.upper() not in data["rates"]: <br>
    return "Invalid currency code. Please try again." <br>

exchange_rate = data["rates"][to_currency.upper()] <br>
converted_amount = amount * exchange_rate <br>

return { <br>
        "converted_amount": converted_amount, <br>
        "exchange_rate": exchange_rate <br>
    } <br>
<br>
tools = [{ <br>
    "type": "function", <br>
    "function": { <br>
        "name": "convert_currency", <br>
        "description": "Converts an amount from one currency to another based on real-time exchange rates.", <br>
        "parameters": { <br>
            "type": "object", <br> 
            "properties": { <br>
                "amount": {"type": "number"}, <br>
                "from_currency": {"type": "string"}, <br>
                "to_currency": {"type": "string"} <br>
            }, <br>
            "required": ["amount", "from_currency", "to_currency"], <br>
            "additionalProperties": False <br>
        }, <br>
        "strict": True <br>
    } <br>
}] <br>

--------------------------------------------------------------
### Initialize OpenAI API caller
openai_caller = OpenAI_LLM_Caller( <br>
    model="gpt-4o-mini", <br>
    api_key=openai_api_key, <br>
    tools=tools <br>
) <br>

user_input = ""<br> 
system_message = """You are a currency conversion assistant. Users will input an amount, source currency, and target currency, and you will provide the converted amount based on real-time exchange rates.\n\nFormat your response as a JSON:\n{\n  \"message\": \"Your converted amount is X\",\n  \"emoji string\": \"💱🌍\"\n}\n\nExample:\n{\n  \"message\": \"100 USD is equivalent to 85 EUR.\",\n  \"emoji string\": \"💵➡️💶\"\n}\n"""
messages = [{"role": "system", "content": system_message}] <br>

print("Chatbot initialized, type in \"exit\" to end the conversation\n")

while user_input.lower() != "exit": <br>
    assistant_response = openai_caller.generate_completion( <br>
        messages=messages, <br>
        response_format={"type": "json_object"} <br>
    ) <br>

if isinstance(assistant_response, ChatCompletionMessage):
        messages.append(assistant_response)
        print("---")
        openai_caller.print_wrap_text(f"assistant tool call: {assistant_response}")

for tool_call in assistant_response.tool_calls:
         if tool_call.function.name == "convert_currency":
            args = json.loads(tool_call.function.arguments)
            result = convert_currency(args["amount"], args["from_currency"], args["to_currency"])
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(result)
            })
            openai_caller.print_wrap_text(f"tool call response: {result}")

print("---\n")
    assistant_response = openai_caller.generate_completion(
        messages=messages,
        response_format={"type": "json_object"}
        )

 openai_caller.print_wrap_text(f"assistant: {assistant_response}")
print("\n")
user_input = input("user: ")
print("\n")
   messages.append({"role": "assistant", "content": assistant_response})
 messages.append({"role": "user", "content": user_input})

print(f"\n----\nTotal Conversation Cost: {openai_caller.total_cost}")
